In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sargas import Potential, Simulation, Configuration, System, Sampler, MCMove, MonteCarlo
import tqdm
import matplotlib.pyplot as plt

sns.set_palette("Dark2")
sns.set_style("ticks")
sns.set_context("talk")

# Assignment 1: Monte Carlo in the NVT ensemble

In this exercise, we will study a 3D Lennard-Jones system in the NVT ensemble, i.e. a system is defined in terms of 

- a cubic box of volume *V*, that
- contains *N* particles,
- at a given temperature *T*.

## Table of Contents <a id="Table-Of-Contents"/>

- [Code Overview](#Code-Overview)
  - [System](#System)
  - [Propagator](#Propagator)
  - [Sampler](#Sampler)
- [Compiling and Testing](#Compiling-And-Testing)
- [Your Tasks](#Your-Tasks)
  - [1. Nearest Image Convention and Periodic Boundary Conditions](#PBC)
  - [2. Run a Simulation](#Simulation)
  - [3. Pressure and Ideal Gas Law](#Pressure)
  - [4. The Truncated and Shifted Lennard-Jones Potential](#Truncated-Shifted)
  - [5. Tail Corrections](#Tail-Corrections)
  - [6. Comparison to NIST](#NIST)

## Code Overview <a id="Code-Overview"/>

[↑ back to top](#Table-Of-Contents)


A **`Simulation`** is built from three parts

- A **`System`**,
- a **`Propagator`**, and possibly
- one or more **`Sampler`**.

<img src="sargas_structure.jpg" width="50%" align="left">

### **`System`** <a id="System"/>

[↑ back to top](#Table-Of-Contents)

A `System` (defined in `src/system.rs`) consists of 

- the `Configuration` (`src/configuration.rs`), in which positions, velocities and forces are stored.
    - the positions, velocities, etc. are stored as `Vec3` which contains an `x`, `y` and `z` component and for which lots of useful mathematical functions are implementend in `src/vec3.rs`.
- And the `Potential` (`src/potential.rs`) which defines the pair energy between two particles as function of the distance between the particles.

### **`Propagator`** <a id="Propagator"/>

[↑ back to top](#Table-Of-Contents)

A `Propagator` takes the current system state (positions, forces, velocities) and generates a new system state. This can be done either using

- `MonteCarlo` (`src/propagator/monte_carlo/mod.rs`) which consists of one or more `Moves` and changes the system state according to the Metropolis acceptance criterion, or
- `MolecularDynamics` (`src/propagator/molecular_dynamics/mod.rs`) which integrates the equations of motion and possibly applies a `Thermostat`.

### **`Sampler`** <a id="Sampler"/>

[↑ back to top](#Table-Of-Contents)

A `Sampler` is an object that defines two things: First, a function that takes the current `System` and calculates or stores something for post-processing (e.g. the system energy or pressure or the positions), and second, a `frequency` that defines how often the function is called.

Examples for `Sampler` are:

- `PropertiesSampler` which stores residual pressure, potential and kinetic energy as well as the virial,
- `WidomSampler` which inserts ghost particles into a configuration to compute the chemical potential (we will implement this in a later assignment),
- `TrajectoryWriter` which dumps the particle positions into a file.

## Compiling and Testing <a id="Compiling-And-Testing"/>

[↑ back to top](#Table-Of-Contents)

To **compile** the code (*which does not build the python module*), use 

```
RUSTFLAGS="-C target-cpu=native" cargo build --release
```

To **test** the code, type

```
RUSTFLAGS="-C target-cpu=native" cargo test --release --test assignment1
```
<div class="alert alert-danger">A red box signals that there is a <strong>test</strong> for an implementation! Run the test with <code>RUSTFLAGS="-C target-cpu=native" cargo test --release --test assignment1</code></div>

To build the **Python module**, type

```
RUSTFLAGS="-C target-cpu=native" maturin develop --release
```

If the last step fails stating that "the resource is currently in use" (or something similar) make sure to **stop the Jupyter kernel** and try running the command again.

---
## Your Tasks <a id="Your-Tasks"/>

[↑ back to top](#Table-Of-Contents)

---

### 1. Nearest Image Convention and Periodic Boundary Conditions. <a id="PBC"/>

[↑ back to top](#Table-Of-Contents)


In `src/vec3.rs` implement the following:

- **Nearest image convention:** when computing the distance between two points in space, consider the nearest image.
    - <div class="alert alert-danger"><strong>Test:</strong> <code>nearest_image_convention</code></div>
- **Periodic boundary condition:** when a particle leaves the simulation volume, it re-enters from the opposite side.
    - <div class="alert alert-danger"><strong>Test:</strong> <code>periodic_boundary_condition</code></div>

---

### 2. Run a Simulation: Equilibration and Production <a id="Simulation"/>

[↑ back to top](#Table-Of-Contents)

Perform a $NVT$ simulation using the following parameters:

**Potential**:

|Parameter|Value|
|-|-|
|Type| Lennard-Jones|
|$\sigma$|1.0|
|$\epsilon$|1.0|
|$r_c$|2.5|
|Shift|No|
|Tail-correction|No|

**Simulation**

|Parameter|Value|
|-|-|
|$N$ | 500 |
|Initial configuration | lattice |
|$T^*$|1.28|
|$\rho^*$|0.9|
|$N_\text{cycles}$|10000|

In [ ]:
nparticles = 
temperature = 
density = 
rc = 
ncycles = 

# Build the system
lennard_jones = Potential.lennard_jones()
configuration = Configuration.lattice()
system = System(configuration=configuration, potential=lennard_jones)

# Build the propagator
particle_displacement = MCMove.displace_particle()
propagator = MonteCarlo()

# Build the simulation
simulation = Simulation.monte_carlo(system, propagator)

# Add sampler
properties = Sampler.properties(500)
simulation.add_sampler(properties)

In [ ]:
# Run the simulation
for _ in tqdm.tqdm(range(ncycles)):
    simulation.run(nparticles)

In [ ]:
data = pd.DataFrame(properties.data)
data

**Write a function,`nvt`, that allows convenient setup of a simulation.**

1. Calculate the average energy of the system at different densities and temperatures.   
2. At what conditions is it easier (faster) to equilibrate the system?

In [ ]:
def nvt(nparticles, temperature, density, nequilibration, nproduction, nsample):
    """Run a NVT simulation.
    
    Parameters
    ----------
    nparticles : int
        number of particles
    temperature : float
        reduced temperature
    density : float
        reduced density
    nequilibration : int
        number of equilibration steps
    nproduction : int
        number of production steps
    nsample : int
        sample after this many steps
        
    Returns
    -------
    pd.DataFrame : data frame containing results
    """
    
    # Set up potential, configuration, propagator and simulation.
    # MODIFY HERE
    simulation = None
    # STOP MODIFY
    
    properties = Sampler.properties(nsample)
    # Run equilibration
    print("Running equilibration ...")
    for _ in tqdm.tqdm(range(nequilibration)):
        simulation.run(nparticles)
    
    # stop adjusting the maximum displacement
    simulation.deactivate_propagator_updates() 
    
    # Run production
    print("Running production ...")
    for _ in tqdm.tqdm(range(nproduction)):
        simulation.run(nparticles)
    
    print("Done.")
    return pd.DataFrame(properties.data)

In [ ]:
temperatures = []
densities = []
results = {}

for t in temperatures:
    for d in densities:
        # run simulation
        data = nvt()
        results.update({(t, d): data})

---

### 3. The Pressure and the Ideal Gas Law <a id="Pressure"/>
    
[↑ back to top](#Table-Of-Contents)

In the present code, the pressure of the system is not calculated correctly. 
You have to implement three things:

1. Implement the `virial`  function in`src/potential.rs` for the Lennard-Jones potential.
2. Implement the `energy_virial`  function in `src/potential.rs` for the Lennard-Jones potential.
3. Implement the residual pressure in the `sample` function of the `PropertiesSampler` in the file: `src/sampler.rs`.

<div class="alert alert-danger"><strong>Test:</strong> <code>lennard_jones_virial</code></div>

For $T^* = 2.0$, perform simulations at different densities. Up to which density does the ideal gas law hold? Note that the pressure in the <code>Sampler.properties()</code> is the residual pressure.

---

### 4. The Truncated and Shifted Lennard-Jones Potential <a id="Truncated-Shifted"/>

[↑ back to top](#Table-Of-Contents)

Up to now, we used the so-called **truncated** Lennard-Jones potential, since we ignored energies above the cut-off distance.
A different widely used form of the Lennard-Jones potential is the **truncated and shifted** potential:

\begin{align}
   U(r) &= 4 \epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right] - 4 \epsilon \left[\left(\frac{\sigma}{r_c}\right)^{12} - \left(\frac{\sigma}{r_c}\right)^6\right] ~\text{if}~ r < r_c \\
   U(r) &= 0.0 ~\text{if}~ r \geq r_c
\end{align}

Implement the energy shift in the function `new_shifted` in `src/potential.rs`.
<div class="alert alert-danger"><strong>Test:</strong> <code>lennard_jones_shift</code></div>

---

### 5. Tail Corrections <a id="Tail-Corrections"/>

[↑ back to top](#Table-Of-Contents)

Cutting-off the potential introduces errors into our simulation since we omit long-range energies.
We can account for long-range contributions to the potential beyond $r_c$ with so-called tail corrections.
For the **truncated** Lennard-Jones potential, we only need one correction:

\begin{align}
  \frac{u_\text{tail}}{N} = \frac{8}{3}\pi\rho\varepsilon\sigma^3\left[\frac{1}{3}\left(\frac{\sigma}{r_c}\right)^9 - \left(\frac{\sigma}{r_c}\right)^3\right] \\
  p_\text{tail} = \frac{16}{3}\pi\rho^2\varepsilon\sigma^3\left[\frac{2}{3}\left(\frac{\sigma}{r_c}\right)^9 - \left(\frac{\sigma}{r_c}\right)^3\right]
\end{align}

wheras for the **truncated and shifted** Lennard-Jones potential, we also have to account for the error we are introducing by shifting the energy. 
This contribution (per particle) reads:

\begin{align}
u_\text{shift-corr} = \frac{1}{2} U(r_c) \langle N \rangle_{r_c},
\end{align}

where $\langle N \rangle_{r_c}$ is the mean particle number within the cut-off radius.  
We would need to know this mean particle number prior to the simulation to evaluate this equation.
For this assignment we use a rough assumption, that is we set the radial distribution function to unity ($g(r)\approx 1$) for which follows

\begin{align}
u_\text{shift-corr} = \frac{2\pi \rho}{3} r_c^3 U(r_c)
\end{align}

Implement the tail correction for the energy and the pressure for the Lennar-Jones potential in `src/potential.rs`.
<div class="alert alert-danger"><strong>Test:</strong> <code>lennard_jones_energy_tail</code></div>
<div class="alert alert-danger"><strong>Test:</strong> <code>lennard_jones_pressure_tail</code></div>

---

### 6. Simulations: Comparison to NIST  <a id="NIST"/>

[↑ back to top](#Table-Of-Contents)

Set up a simulation according to the [NIST Lennard-Jones simulation benchmarks](https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm).

1. Perform a simulation for a liquid and a vapor phase (choose the states from the table below).
2. Compare the results. To estimate the error, compute the autocorrelation time.

|T*	|ρ*	|U*	|+/-|	p*	|+/-|
|-|-|-|-|-|-|
| 8.50E-01 | 1.00E-03 | -1.0317E-02 | 2.34E-05 | 8.4402E-04 | 4.66E-08 |
| 8.50E-01 | 3.00E-03 | -3.1019E-02 | 5.91E-05 | 2.4965E-03 | 4.99E-07 |
| 8.50E-01 | 5.00E-03 | -5.1901E-02 | 7.53E-05 | 4.1003E-03 | 5.05E-07 |
| 8.50E-01 | 7.00E-03 | -7.2834E-02 | 1.34E-04 | 5.6565E-03 | 7.96E-07 |
| 8.50E-01 | 9.00E-03 | -9.3973E-02 | 1.29E-04 | 7.1641E-03 | 2.24E-06 |
| 8.50E-01 | 7.76E-01 | -5.5121E+00 | 4.55E-04 | 6.7714E-03 | 1.77E-03 |
| 8.50E-01 | 7.80E-01 | -5.5386E+00 | 7.26E-04 | 4.7924E-02 | 3.18E-03 |
| 8.50E-01 | 8.20E-01 | -5.7947E+00 | 6.03E-04 | 5.5355E-01 | 4.13E-03 |
| 8.50E-01 | 8.60E-01 | -6.0305E+00 | 2.38E-03 | 1.2660E+00 | 1.36E-02 |
| 8.50E-01 | 9.00E-01 | -6.2391E+00 | 5.27E-03 | 2.2314E+00 | 2.72E-02 |
| 9.00E-01 | 1.00E-03 | -9.9165E-03 | 1.89E-05 | 8.9429E-04 | 2.48E-08 |
| 9.00E-01 | 3.00E-03 | -2.9787E-02 | 3.21E-05 | 2.6485E-03 | 2.54E-07 |
| 9.00E-01 | 5.00E-03 | -4.9771E-02 | 3.80E-05 | 4.3569E-03 | 2.19E-07 |
| 9.00E-01 | 7.00E-03 | -6.9805E-02 | 7.66E-05 | 6.0193E-03 | 1.02E-06 |
| 9.00E-01 | 9.00E-03 | -8.9936E-02 | 2.44E-05 | 7.6363E-03 | 1.44E-06 |
| 9.00E-01 | 7.76E-01 | -5.4689E+00 | 4.20E-04 | 2.4056E-01 | 2.74E-03 |
| 9.00E-01 | 7.80E-01 | -5.4956E+00 | 7.86E-04 | 2.7851E-01 | 2.97E-03 |
| 9.00E-01 | 8.20E-01 | -5.7456E+00 | 7.51E-04 | 8.2386E-01 | 2.85E-03 |
| 9.00E-01 | 8.60E-01 | -5.9753E+00 | 5.53E-04 | 1.5781E+00 | 3.29E-03 |
| 9.00E-01 | 9.00E-01 | -6.1773E+00 | 1.57E-03 | 2.5848E+00 | 9.54E-03 |